In [ ]:
import os
import pandas as pd
import numpy as np
import networkx as nx
from tqdm import tqdm

def extrair_caracteristicas_grafo_lista_adjacencia(df, nome_instancia):
    # Garantir que 'Vértice' seja do tipo inteiro
    df['Vértice'] = df['Vértice'].astype(int)

    # Inicializar o grafo
    G = nx.Graph()

    # Adicionar vértices
    for vertice in df['Vértice']:
        G.add_node(vertice)

    # Adicionar arestas com pesos
    for index, row in df.iterrows():
        vertice = row['Vértice']
        adjacentes_str = row['Adjacentes']
        if pd.isna(adjacentes_str) or adjacentes_str.strip() == '':
            continue  # Vértice sem adjacentes
        adjacentes = adjacentes_str.split(';')
        for adj in adjacentes:
            adj = adj.strip()
            if adj:
                try:
                    vizinho, peso = adj.split('(')
                    vizinho = int(vizinho.strip())
                    peso = float(peso.strip(')'))
                    G.add_edge(vertice, vizinho, weight=peso)
                except ValueError:
                    print(f"⚠️ Formato inválido na adjacência: '{adj}' para o vértice {vertice}")

    # Verificar se o grafo foi construído corretamente
    if len(G.nodes) == 0:
        print(f"⚠️ Grafo {nome_instancia} vazio. Nenhum vértice foi adicionado.")
    if len(G.edges) == 0:
        print(f"⚠️ Grafo {nome_instancia} sem arestas. Nenhuma aresta foi adicionada.")

    # Extrair 'Conjuntos' por vértice
    if 'Conjuntos' not in df.columns:
        print(f"⚠️ Coluna 'Conjuntos' não encontrada no DataFrame de entrada para a instância {nome_instancia}.")
        conjuntos_dict = {v: np.nan for v in G.nodes}
    else:
        conjuntos_series = df.set_index('Vértice')['Conjuntos']
        conjuntos_dict = conjuntos_series.to_dict()

    # Calcular características por vértice
    graus = dict(G.degree(weight='weight'))
    soma_pesos = {v: sum(d['weight'] for _, _, d in G.edges(v, data=True)) for v in G.nodes}
    pesos_arestas = {v: [d['weight'] for _, _, d in G.edges(v, data=True)] for v in G.nodes}
    peso_medio = {v: np.mean(pesos) if pesos else 0 for v, pesos in pesos_arestas.items()}
    desvio_peso = {v: np.std(pesos) if pesos else 0 for v, pesos in pesos_arestas.items()}
    peso_max = {v: max(pesos) if pesos else 0 for v, pesos in pesos_arestas.items()}
    peso_min = {v: min(pesos) if pesos else 0 for v, pesos in pesos_arestas.items()}
    densidade_local = nx.clustering(G, weight='weight')

    # Criar DataFrame de características
    df_caracteristicas = pd.DataFrame({
        'Vértice': list(G.nodes),
        'Grau (Ponderado)': [graus.get(v, 0) for v in G.nodes],
        'Soma dos Pesos das Arestas': [soma_pesos.get(v, 0) for v in G.nodes],
        'Peso Médio das Arestas': [peso_medio.get(v, 0) for v in G.nodes],
        'Desvio Padrão dos Pesos': [desvio_peso.get(v, 0) for v in G.nodes],
        'Peso Máximo das Arestas': [peso_max.get(v, 0) for v in G.nodes],
        'Peso Mínimo das Arestas': [peso_min.get(v, 0) for v in G.nodes],
        'Densidade Local': [densidade_local.get(v, 0) for v in G.nodes],
        'Conjuntos': [conjuntos_dict.get(v, np.nan) for v in G.nodes],
    })

    # Remover a coluna 'Conjuntos' do DataFrame original para evitar duplicação
    df = df.drop(columns=['Conjuntos'])

    # Combinar com a lista de adjacência original
    df_final = df.merge(df_caracteristicas, on='Vértice', how='left')

    # Reorganizar as colunas: 'Instância' primeiro, 'Vértice', 'Adjacentes', características..., 'Conjuntos' por último
    colunas = df_final.columns.tolist()

    # Remover 'Instância' e 'Conjuntos' da lista original para reorganizar
    colunas.remove('Conjuntos')

    # Definir a nova ordem das colunas
    nova_ordem = colunas + ['Conjuntos']

    # Aplicar a nova ordem
    df_final = df_final[nova_ordem]

    return df_final

def processar_pasta_grafos_lista_adjacencia(pasta_entrada, pasta_saida):
    # Garantir que a pasta de saída exista
    os.makedirs(pasta_saida, exist_ok=True)

    # Listar todos os arquivos CSV na pasta de entrada
    arquivos = [f for f in os.listdir(pasta_entrada) if f.endswith('.csv')]

    if not arquivos:
        print(f"❌ Nenhum arquivo CSV encontrado na pasta de entrada: {pasta_entrada}")
        return

    for arquivo in tqdm(arquivos, desc="Processando grafos"):
        caminho_arquivo = os.path.join(pasta_entrada, arquivo)
        nome_instancia_original = os.path.splitext(arquivo)[0]
        # Formatar o nome da instância removendo '_lista_adjacencia'
        nome_instancia = nome_instancia_original.replace('_lista_adjacencia', '')

        try:
            # Ler o arquivo CSV da lista de adjacência
            df = pd.read_csv(caminho_arquivo)
            print(f"📄 Lendo arquivo: {arquivo}")
        except Exception as e:
            print(f"❌ Erro ao ler o arquivo {arquivo}: {e}")
            continue

        # Verificar se as colunas necessárias estão presentes
        if not {'Vértice', 'Adjacentes', 'Conjuntos'}.issubset(df.columns):
            print(f"❌ Arquivo {arquivo} não possui as colunas necessárias. Colunas encontradas: {df.columns.tolist()}")
            continue

        # Extrair características e combinar com a lista de adjacência
        df_final = extrair_caracteristicas_grafo_lista_adjacencia(df, nome_instancia)

        if df_final.empty:
            print(f"⚠️ Arquivo {arquivo}: DataFrame final está vazio.")
            continue

        # Definir o nome do arquivo de saída
        nome_saida = os.path.join(pasta_saida, f"{nome_instancia}_caracteristicas.csv")

        try:
            # Salvar o DataFrame final
            df_final.to_csv(nome_saida, index=False)
            print(f"✅ Salvo: {nome_saida}")
        except Exception as e:
            print(f"❌ Erro ao salvar o arquivo {nome_saida}: {e}")

if __name__ == "__main__":
    # Informações de início
    print("🚀 Iniciando o processamento dos grafos...")

    # Definir os caminhos das pastas (Atualize conforme a sua estrutura de diretórios)
    # Utilize caminhos relativos ou absolutos conforme necessário
    # Exemplo usando caminhos relativos:
    pasta_entrada = "New Instances Carlos/Uniformes Inteiros Lista Adj"  # Atualize para o caminho correto
    pasta_saida = "New Instances Carlos/Uniformes Inteiros Lista Adj Extraidas"  # Atualize para o caminho correto

    print(f"Pasta de entrada: {pasta_entrada}")
    print(f"Pasta de saída: {pasta_saida}")

    # Imprimir diretório atual de execução
    print(f"Diretório Atual de Execução: {os.getcwd()}")

    # Processar os grafos
    processar_pasta_grafos_lista_adjacencia(pasta_entrada, pasta_saida)

    print("🎉 Processamento concluído!")
